In [ ]:
# Enter details of your AML workspace
subscription_id = "b3b0e63c-e8fd-4f5c-bab9-1ed82844ef1f"
resource_group = "jameschao"
workspace = "jameschao"
compute_name = "cpu-cluster"

# Enter details of your model and components
version_string = "2"
data_version = "1"
model_id = "azureml_plucky_shelf_lgrt16cy1f_40_output_mlflow_log_model_261482756:1"

In [ ]:
import time

from azure.ai.ml import Input, MLClient, Output, dsl, load_job
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
from azure.ai.ml.constants import AssetTypes
from azure.identity import DefaultAzureCredential

In [ ]:
# Create MLClient object

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    workspace_name=workspace,
)
print(ml_client)

# Get handle to azureml registry for the RAI built in components

registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

In [ ]:
# Helper function


def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job

In [ ]:
def forecasting_rai_dashboard():

    rai_constructor_component = ml_client.components.get(
        name="rai_insights_constructor", version=version_string
    )

    rai_gather_component = ml_client.components.get(
        name="rai_insights_gather", version=version_string
    )

    input_train_data = "forecasting_energy_train_mltable"
    input_test_data = "forecasting_energy_test_mltable"

    train_data = Input(
        type="mltable",
        path=f"azureml:{input_train_data}:{data_version}",
        mode="download",
    )
    test_data = Input(
        type="mltable",
        path=f"{input_test_data}:{data_version}",
        mode="download",
    )

    # Pipeline skips on analysis; relies on the constructor component verifying the model works
    @dsl.pipeline(
        compute=compute_name,
        description="Test of Create RAI Dashboard for Forecasting",
        experiment_name=f"test_create_rai_dashboard_forecasting{version_string}",
    )
    def construct_dashboard(train_data, test_data):
        construct_job = rai_constructor_component(
            title="Run built from DSL",
            task_type="forecasting",
            model_info=model_id,
            model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=f"azureml:{model_id}"),
            train_dataset=train_data,
            test_dataset=test_data,
            target_column_name="demand",
            feature_metadata='{"datetime_features": ["datetime"], '
            '"time_series_id_features": ["group_id"]}',
            categorical_column_names='["precip", "temp", "group_id"]',
            maximum_rows_for_test_dataset=5000,
            classes="[]",  # Should be default value
            use_model_dependency=True,
        )
        construct_job.set_limits(timeout=1800)

        rai_gather_job = rai_gather_component(
            constructor=construct_job.outputs.rai_insights_dashboard
        )
        rai_gather_job.set_limits(timeout=1800)

    insights_pipeline_job = construct_dashboard(
        train_data=train_data,
        test_data=test_data,
    )

    # Send it
    insights_pipeline_job = submit_and_wait(ml_client, insights_pipeline_job)
    assert insights_pipeline_job is not None

In [ ]:
forecasting_rai_dashboard()